In [25]:
import requests
import json
from collections import defaultdict

# GitHub repository information
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'  # Or any other branch you want to fetch from

# GitHub API endpoint to list files in the repository
api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'

# Alias mapping (key: alias, value: primary name)
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'Ashalo1': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}

def get_primary_name(name):
    """Returns the primary name for a given player name using the alias mapping."""
    return player_aliases.get(name, name)  # Return primary name if alias exists, otherwise the name itself

def fetch_files_from_github(api_url, path=''):
    """Fetches JSON files from the given GitHub repository path."""
    files = []
    response = requests.get(f'{api_url}/{path}')
    if response.status_code == 200:
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                files.append(item['download_url'])
            elif item['type'] == 'dir':
                files.extend(fetch_files_from_github(api_url, item['path']))
    else:
        print(f"Failed to fetch files: {response.status_code} - {response.text}")
    return files

def fetch_json_from_url(url):
    """Fetches JSON data from the given URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch JSON from {url}: {response.status_code} - {response.text}")
    return None

# Fetch JSON files from the GitHub repository
json_files_urls = fetch_files_from_github(api_url)

# Initialize player stats dictionary
player_stats = defaultdict(lambda: {
    'wins': 0,
    'losses': 0,
    'kills': 0,
    'deaths': 0,
    'assists': 0,
    'matches': 0,
    'total_kills': 0,
    'penta_kills': 0,
    'games_played': 0
})

# Parse and extract data from each JSON file
for file_url in json_files_urls:
    match = fetch_json_from_url(file_url)
    if match:
        #print(f"Processing file: {file_url}")  # Log file being processed
        if 'participants' not in match:
            print(f"No participants data in file: {file_url}")
            continue
        participants = match['participants']
        for player in participants:
            if 'NAME' not in player:
                print(f"No player name in file: {file_url}")
                continue
            player_name = get_primary_name(player['NAME'])  # Use primary name
            player_stats[player_name]['matches'] += 1
            player_stats[player_name]['kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['assists'] += int(player.get('ASSISTS', 0))
            player_stats[player_name]['deaths'] += int(player.get('NUM_DEATHS', 0))
            player_stats[player_name]['total_kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['penta_kills'] += int(player.get('PENTA_KILLS', 0))
            player_stats[player_name]['games_played'] += 1
            
            # Check win/loss
            if player.get('WIN') == 'Win':
                player_stats[player_name]['wins'] += 1
            else:
                player_stats[player_name]['losses'] += 1

if not player_stats:
    print("No player data found.")
else:
    # Compute win rate and average kills per match
    for player_name, stats in player_stats.items():
        stats['win_rate'] = stats['wins'] / stats['matches'] * 100
        stats['avg_kills_per_match'] = stats['kills'] / stats['matches']

    # Sort players by the number of matches played, descending
    sorted_players = sorted(player_stats.items(), key=lambda x: x[1]['matches'], reverse=True)   
    
    # Find player with the most deaths
    most_deaths_player = max(player_stats.items(), key=lambda x: x[1]['deaths'])
    most_kills_player = max(player_stats.items(), key=lambda x: x[1]['kills'])
    print(f"Player with the most deaths: {most_deaths_player[0]} with {most_deaths_player[1]['deaths']} deaths")
    print(f"Player with the most kills: {most_kills_player[0]} with {most_kills_player[1]['kills']} kills")
    
    # Print statistics for each player
    for player_name, stats in sorted_players:
        print(f"Player: {player_name}")
        print(f"  Win Rate: {stats['win_rate']:.2f}%")
        print(f"  Games Played: {stats['games_played']}")
        #print(f"  Total Kills: {stats['total_kills']}")
        #print(f"  Total Deaths: {stats['deaths']}")
        print(f"  Average Kills per Match: {stats['avg_kills_per_match']:.2f}")
        avg_kda=(stats['kills']+stats['assists'])/stats['deaths']
        avg_kda=round(avg_kda, 2)
        print(f"  Average KDA: {avg_kda}")
        #print(f" Pentakills: {stats['penta_kills']}")



        print()


Player with the most deaths: IbelieveMeYou with 287 deaths
Player with the most kills: adostraa with 400 kills
Player: nigeriacoc
  Win Rate: 54.39%
  Games Played: 57
  Average Kills per Match: 5.11
  Average KDA: 3.0

Player: adostraa
  Win Rate: 57.14%
  Games Played: 56
  Average Kills per Match: 7.14
  Average KDA: 3.51

Player: TylerToTheT212
  Win Rate: 71.15%
  Games Played: 52
  Average Kills per Match: 7.63
  Average KDA: 4.11

Player: IbelieveMeYou
  Win Rate: 44.00%
  Games Played: 50
  Average Kills per Match: 6.82
  Average KDA: 2.43

Player: Ashalo1
  Win Rate: 41.30%
  Games Played: 46
  Average Kills per Match: 6.41
  Average KDA: 2.6

Player: ihyash
  Win Rate: 52.63%
  Games Played: 38
  Average Kills per Match: 2.95
  Average KDA: 3.99

Player: DonutDude17
  Win Rate: 67.65%
  Games Played: 34
  Average Kills per Match: 3.71
  Average KDA: 2.75

Player: GasterBat
  Win Rate: 67.74%
  Games Played: 31
  Average Kills per Match: 4.00
  Average KDA: 2.33

Player: Bryan